*   Paper: "High Quality Monocular Depth Estimation via Transfer Learning"
*   Author: Ibraheem Alhashim
*   University: KAUST
*   Country: Saudi Arabia

# Clone images repository

Loading a scene

In [ ]:
# Setting input folder
scene_name = '/validation'
input_dir = '/content' + scene_name
github_path = 'https://gitlab.com/victorvirgilio' + scene_name + '.git'
# Setting output folder
!mkdir output
!mkdir output/png_images
!mkdir output/npy_images
!mkdir output/npy_errors
output_dir = '/content/output'
# Save?
save = True

!git clone $github_path $input_dir

Cloning into '/content/validation'...
remote: Enumerating objects: 14976, done.
remote: Counting objects: 100% (14976/14976), done.
remote: Compressing objects: 100% (14976/14976), done.
remote: Total 14976 (delta 1), reused 14970 (delta 0), pack-reused 0
Receiving objects: 100% (14976/14976), 5.68 GiB | 48.17 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Checking out files: 100% (22446/22446), done.


# Clone github repository

In [ ]:
# Es necesario descargar los archivos útiles a mi propio dominio.

!git clone https://github.com/ialhashim/DenseDepth.git
%cd DenseDepth

Cloning into 'DenseDepth'...
remote: Enumerating objects: 222, done.
remote: Total 222 (delta 0), reused 0 (delta 0), pack-reused 222
Receiving objects: 100% (222/222), 11.79 MiB | 38.46 MiB/s, done.
Resolving deltas: 100% (106/106), done.
/content/DenseDepth


# Download KITTI pretrained model

In [ ]:
model_name = 'kitti.h5'
model_path = 'https://s3-eu-west-1.amazonaws.com/densedepth/' + model_name

# Es necesario descargar el modelo a mi propio dominio.
!wget $model_path -O $model_name

--2020-02-17 16:43:56--  https://s3-eu-west-1.amazonaws.com/densedepth/kitti.h5
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.36.66
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.36.66|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172897384 (165M) [application/x-www-form-urlencoded]
Saving to: ‘kitti.h5’

kitti.h5            100%[===================>] 164.89M  31.3MB/s    in 5.9s    

2020-02-17 16:44:03 (27.8 MB/s) - ‘kitti.h5’ saved [172897384/172897384]



# Load KITTI pretrained model

In [ ]:
# Hace falta instalar versión para evitar warnings.

import os
import time
import math
import glob
import argparse
import matplotlib
import numpy as np
from PIL import Image
from skimage.transform import resize

# Keras / TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
from keras.models import load_model
from layers import BilinearUpSampling2D
from utils import predict, load_images, display_images
from matplotlib import pyplot as plt

# Custom object needed for inference and training
custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': None}
print('\n\tLoading model...\n')
# Load model into GPU / CPU
model = load_model(model_name, custom_objects=custom_objects, compile=False)
print('\n\t\tModel loaded ({0}).'.format(model_name))

Using TensorFlow backend.



	Loading model...
















		Model loaded (kitti.h5).


# Load RGB and Ground Truth images.

In [ ]:
def load_rgb_images(image_files):
    loaded_images = []
    for file in image_files:     
        img = Image.open( file )
        img = img.resize((1280, 384), Image.NEAREST)
        x = np.clip(np.asarray(img, dtype=float) / 255, 0, 1)
        loaded_images.append(x)
    return np.stack(loaded_images, axis=0)

def load_depth_maps(image_files):
    loaded_images = []
    for file in image_files:  
      depth_png = np.array(Image.open(file), dtype=int)
      depth = depth_png.astype(np.float) / 256.
      depth[depth_png == 0] = -1.
      loaded_images.append(depth)
    return np.stack(loaded_images, axis=0)

def fit_gt_pred(pred):
  height, width, channels = pred.shape
  pred_reshape = pred.reshape((height, width))
  return resize(pred_reshape, [375, 1242], preserve_range=True, mode='reflect', anti_aliasing=True )


def compute_errors(gt, pred):
    # a
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25   ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()
    # abs_rel
    abs_rel = np.mean(np.abs(gt - pred) / gt)
    # rmse
    rmse = (gt - pred) ** 2
    rmse = np.sqrt(rmse.mean())
    # log_10
    log_10 = (np.abs(np.log10(gt)-np.log10(pred))).mean()
    # sq_rel
    sq_rel = np.mean(((gt - pred)**2) / gt)
    # rmse_log
    rmse_log = (np.log(gt) - np.log(pred)) ** 2
    rmse_log = np.sqrt(rmse_log.mean())
    return a1, a2, a3, abs_rel, rmse, log_10, sq_rel, rmse_log

# Compute and adjust predictions

In [ ]:
# Cargar nombres
names = []
with open('/content/validation/val.txt','r') as inf:
    for line in inf:
        names.append(input_dir + '/' + line[:-1] + '.png')
names = names
print(names)      
print(len(names))  
# print(names[0][:-10] + rgb_names[0][20:-10] + names[0][-10:])

['/content/validation/000001.png', '/content/validation/000002.png', '/content/validation/000004.png', '/content/validation/000005.png', '/content/validation/000006.png', '/content/validation/000008.png', '/content/validation/000015.png', '/content/validation/000019.png', '/content/validation/000020.png', '/content/validation/000021.png', '/content/validation/000023.png', '/content/validation/000024.png', '/content/validation/000025.png', '/content/validation/000027.png', '/content/validation/000028.png', '/content/validation/000031.png', '/content/validation/000033.png', '/content/validation/000035.png', '/content/validation/000037.png', '/content/validation/000039.png', '/content/validation/000040.png', '/content/validation/000042.png', '/content/validation/000047.png', '/content/validation/000048.png', '/content/validation/000050.png', '/content/validation/000052.png', '/content/validation/000053.png', '/content/validation/000058.png', '/content/validation/000059.png', '/content/val

In [ ]:
from utils import display_images


rgb_names = glob.glob(input_dir + '/image_2/*.png')
gt_names = glob.glob(input_dir + '/image_2/*.png')

rgb_names = sorted(rgb_names)[:]
# gt_names = sorted(gt_names)[:25]

new_list = []
for i in names:
  new_list.append('/content/validation/image_2/' + i[-10:])
# rgb_names = new_list

images_num = len(rgb_names)
batch = 5
iterations = math.floor(images_num/batch)
rem_images = images_num - (batch * iterations)


for step in range(iterations + 1):

  if(step != iterations):
    start = step * batch
    end = (step + 1) * batch
  elif(step == iterations and rem_images != 0):
    start = step * batch
    end = start + rem_images
  else:
    break

  print('\n\t Processing: {} - {} \t of {} images'.format(start+1, end, images_num))

  rgb = new_list[start:end]
  inputs = load_rgb_images( rgb )
  # gt = load_depth_maps( gt_names[start:end] )

  # Compute depth maps
  output_maps = predict(model, inputs)
  
  images = []
  for item in range(len(rgb)):
    # Resize and reshape image
    pred = fit_gt_pred(output_maps[item])
    print()
    print(output_maps[item].shape)
    print(pred.shape)
    # # Compute error values
    # mask = gt[item] > 0
    # error = compute_errors(gt[item][mask] / 80, pred[mask])
    # Save predictions
    # Inverse Map
    pred_inv = 1 / pred
    depth_map_rgb = display_images(pred_inv[np.newaxis, ..., np.newaxis].copy())
    img = Image.fromarray(np.uint8(depth_map_rgb * 255))
    img.save(output_dir + '/png_images' + rgb[item][-11:])
    print('\nRecorded image {}!'.format(output_dir + '/png_images' + rgb[item][-11:]))
    if(save):
      # Save each image as .npy 
      np.save(output_dir + '/npy_images' + rgb[item][-11:-4], pred)
      print('Recorded prediction.npy!')
      # # Save each error as .npy
      # np.save(output_dir + '/npy_errors' + rgb[item][-15:-4], error)
      # print('Recorded error.npy!')

print('\n\t {} processed images'.format(images_num))

Se han truncado las últimas 5000 líneas del flujo de salida.
(192, 640, 1)
(375, 1242)

Recorded image /content/output/png_images/005996.png!
Recorded prediction.npy!

(192, 640, 1)
(375, 1242)

Recorded image /content/output/png_images/005997.png!
Recorded prediction.npy!

	 Processing: 2991 - 2995 	 of 7481 images

(192, 640, 1)
(375, 1242)

Recorded image /content/output/png_images/005999.png!
Recorded prediction.npy!

(192, 640, 1)
(375, 1242)

Recorded image /content/output/png_images/006001.png!
Recorded prediction.npy!

(192, 640, 1)
(375, 1242)

Recorded image /content/output/png_images/006002.png!
Recorded prediction.npy!

(192, 640, 1)
(375, 1242)

Recorded image /content/output/png_images/006003.png!
Recorded prediction.npy!

(192, 640, 1)
(375, 1242)

Recorded image /content/output/png_images/006005.png!
Recorded prediction.npy!

	 Processing: 2996 - 3000 	 of 7481 images

(192, 640, 1)
(375, 1242)

Recorded image /content/output/png_images/006008.png!
Recorded prediction.n

ValueError: ignored

# Compute errors

In [ ]:
def load_errors(names):
  errors = []
  for item in names:
    errors.append(np.load(item))
  return np.stack(errors, axis=0)

errors = load_errors( glob.glob(output_dir + '/npy_errors/*.npy') )
e = np.mean(errors, axis=0)

print('\n\tImages:   {}\t\tErrors:   {}'.format(errors.shape[0], errors.shape[1]))

np.save('/content/errors.npy', errors)
print('\n\tRecorded error.npy!')

print('\n\n\t\t\t\t***** METRICS *****\n')
print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format('a1', 'a2', 'a3', 'rel', 'rmse', 'log_10', 'sq_rel', 'rmse_log'))
print("{:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}\n".format(e[0],e[1],e[2],e[3],e[4],e[5],e[6],e[7]))

# Display predictions

In [ ]:
print(output_dir)

/content/output


In [ ]:
# Generate Compressed File (predictions_png.zip)
import zipfile
fantasy_zip = zipfile.ZipFile('/content/predictions_npy.zip', 'w') # Nombre del archivo comprimido
for folder, subfolders, files in os.walk(output_dir):
  for file in files:
    if file.endswith('npy'): # Extensión de los archivos a comprimir
      fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), output_dir), compress_type = zipfile.ZIP_DEFLATED)
fantasy_zip.close()
print('Compressed File generated (predictions_png.zip)!')

Compressed File generated (predictions_png.zip)!


# Files to download ...
*   /content/predictions_png.zip
*   /content/predictions.npy
*   /content/errors.npy

  Note: Rewrite the error values in a table.

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
!cp /content/predictions_npy.zip '/gdrive/My Drive/'